In [ ]:
import geopandas as gpd
import rasterio
from rasterio.warp import calculate_default_transform, reproject, Resampling
from rasterio.mask import mask
from rasterstats import zonal_stats
import numpy as np
import pandas as pd
import os

# ===== Parameter settings =====
gdp_dir = 'GDP_SSP/'          # Folder containing GDP tif files
pop_dir = 'gridded_pop/'      # Folder containing gridded population density tif files
shp_path = 'shi_en.shp'
output_dir = 'weighted_gdp/'  # Output folder for results

years = [2030, 2040, 2050, 2060]

ssp_rcp_map = {
    'ssp1': 'RCP2_6',
    'ssp2': 'RCP4_5',
    'ssp5': 'RCP8_5'
}

fer_scenario = 'mid'  # Fixed as 'mid' scenario
# ==============================

# Read shapefile and use its CRS as the target CRS
cities = gpd.read_file(shp_path)
target_crs = cities.crs


def reproject_raster(in_path, out_path, dst_crs):
    """Reproject a raster to a target CRS and save to out_path."""
    with rasterio.open(in_path) as src:
        transform, width, height = calculate_default_transform(
            src.crs, dst_crs, src.width, src.height, *src.bounds
        )
        kwargs = src.meta.copy()
        kwargs.update({'crs': dst_crs, 'transform': transform, 'width': width, 'height': height})

        with rasterio.open(out_path, 'w', **kwargs) as dst:
            for i in range(1, src.count + 1):
                reproject(
                    source=rasterio.band(src, i),
                    destination=rasterio.band(dst, i),
                    src_transform=src.transform,
                    src_crs=src.crs,
                    dst_transform=transform,
                    dst_crs=dst_crs,
                    resampling=Resampling.nearest
                )
    return out_path


def clip_raster(raster_path, gdf, out_path):
    """Clip a raster by the geometries in gdf and save to out_path."""
    with rasterio.open(raster_path) as src:
        out_image, out_transform = mask(src, gdf.geometry, crop=True)
        out_meta = src.meta.copy()
        out_meta.update({
            'height': out_image.shape[1],
            'width': out_image.shape[2],
            'transform': out_transform
        })

        with rasterio.open(out_path, 'w', **out_meta) as dest:
            dest.write(out_image)
    return out_path


# Main loop
all_results = []
for ssp, rcp in ssp_rcp_map.items():
    for year in years:

        gdp_file = f'GDP{year}_{ssp}.tif'
        pop_file = f'grid_pop_count{year}_{fer_scenario}_{rcp}_new.tif'

        gdp_path = os.path.join(gdp_dir, gdp_file)
        pop_path = os.path.join(pop_dir, pop_file)

        if not os.path.exists(gdp_path) or not os.path.exists(pop_path):
            continue

        # Reproject
        gdp_proj = os.path.join(output_dir, f'gdp_{year}_{ssp}_reproj.tif')
        pop_proj = os.path.join(output_dir, f'pop_{year}_{rcp}_reproj.tif')
        reproject_raster(gdp_path, gdp_proj, target_crs)
        reproject_raster(pop_path, pop_proj, target_crs)

        # Clip
        gdp_clip = os.path.join(output_dir, f'gdp_{year}_{ssp}_clip.tif')
        pop_clip = os.path.join(output_dir, f'pop_{year}_{rcp}_clip.tif')
        clip_raster(gdp_proj, cities, gdp_clip)
        clip_raster(pop_proj, cities, pop_clip)

        # Zonal statistics
        stats = zonal_stats(
            vectors=cities,
            raster=gdp_clip,
            stats=['mean'],
            weight_raster=pop_clip,
            add_stats={'pop_weighted_gdp': lambda v, w: np.average(v, weights=w)},
            geojson_out=False,
            nodata=-9999
        )

        df = pd.DataFrame(stats)
        df['year'] = year
        df['ssp'] = ssp
        df['rcp'] = rcp
        df['city'] = cities['English'] if 'English' in cities.columns else cities.index
        all_results.append(df)

        csv_out = os.path.join(output_dir, f'city_pop_weighted_gdp_{year}_{ssp}_mid_{rcp}.csv')
        df.to_csv(csv_out, index=False)
        print(f'✅ Saved: {csv_out}')
